In [21]:
# Importing libraries for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
# Loading the csv file to get the data
df = pd.read_csv("../datasets/german_credit_data.csv",sep=',',index_col=0)
df.tail()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad
999,27,male,2,own,moderate,moderate,4576,45,car,good


In [ ]:
# Checking the null values
df.shape, df.isnull().sum()
# Out of 1000 rows,
# 183 null values for Saving accounts,
# 394 null values for Checking account
df["Checking account"].unique()
# Check how the missing values are spread
# to figure out a way to clean these values
test = pd.crosstab(df["Checking account"],df["Saving accounts"],dropna=False)
test

Saving accounts,little,moderate,quite rich,rich,NaN
Checking account,,,,,
little,219,12,8,6,29
moderate,152,47,11,14,45
rich,41,5,4,3,10
NaN,191,39,40,25,99
